In [73]:
import pandas as pd

In [74]:
gender = "women"

df = pd.read_csv(f"data/100m_{gender}.csv")
df.head()

,Mark,WIND,Competitor,Nat,Venue,Date
0,10.64,12.0,Carmelita JETER,USA,Shanghai (CHN),2009-09-20
1,10.70,20.0,Carmelita JETER,USA,"Hayward Field, Eugene, OR (USA)",2011-06-04
2,10.70,6.0,Shelly-Ann FRASER-PRYCE,JAM,Kingston (JAM),2012-06-29
3,10.70,3.0,Elaine THOMPSON,JAM,Kingston (JAM),2016-07-01
4,10.71,1.0,Shelly-Ann FRASER-PRYCE,JAM,"Khalifa International Stadium, Doha (QAT)",2019-09-29


In [75]:
# drop multiple personal bests over the years and keep just the highest one for each competitor
df = df.sort_values("Mark").drop_duplicates(subset=["Competitor"])
# correct error in wind speed
df["WIND"] = df.WIND/10
df.head()

,Mark,WIND,Competitor,Nat,Venue,Date
0,10.64,1.2,Carmelita JETER,USA,Shanghai (CHN),2009-09-20
2,10.70,0.6,Shelly-Ann FRASER-PRYCE,JAM,Kingston (JAM),2012-06-29
3,10.70,0.3,Elaine THOMPSON,JAM,Kingston (JAM),2016-07-01
10,10.74,1.0,English GARDNER,USA,"Hayward Field, Eugene, OR (USA)",2016-07-03
11,10.75,0.4,Kerron STEWART,JAM,"Stadio Olimpico, Roma (ITA)",2009-07-10


In [76]:
df["Date"] = pd.to_datetime(df.Date)
df["year"] = df.Date.dt.year


df = df[df.Mark <= min(df.groupby("year").Mark.max())]

In [77]:
# smooth duplicate Marks and convert data to speed
for mark, mark_df in df.groupby("Mark"):
    smoothed_values = [(mark-0.005) + 0.01*((2*(i+1)-1)/(2*len(mark_df))) for i in range(len(mark_df))]
    df.loc[mark_df.index, "Mark"] = smoothed_values
    
df["Speed"] = 100/df.Mark * 3.6
df.head()

,Mark,WIND,Competitor,Nat,Venue,Date,year,Speed
0,10.6400,1.2,Carmelita JETER,USA,Shanghai (CHN),2009-09-20,2009,33.834586
2,10.6975,0.6,Shelly-Ann FRASER-PRYCE,JAM,Kingston (JAM),2012-06-29,2012,33.652723
3,10.7025,0.3,Elaine THOMPSON,JAM,Kingston (JAM),2016-07-01,2016,33.637001
10,10.7400,1.0,English GARDNER,USA,"Hayward Field, Eugene, OR (USA)",2016-07-03,2016,33.519553
11,10.7475,0.4,Kerron STEWART,JAM,"Stadio Olimpico, Roma (ITA)",2009-07-10,2009,33.496162


In [78]:
df.drop("year", axis=1).to_csv(f"data/100m_{gender}_filtered.csv", index=False)